In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import string
import datetime

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['dev_server']
sladb = config['db']['sladb']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + sladb + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Write all of the sql queries required for the tests

In [5]:
sql_sla_season = 'select * from sladb.dbo.tbl_sla_season'

In [7]:
sql_change_request = 'select * from sladb.dbo.tbl_change_request'

In [8]:
sql_change_request_status = 'select * from sladb.dbo.tbl_change_request_status'

In [9]:
sql_unit_sla_season = 'select * from sladb.dbo.tbl_unit_sla_season'

In [10]:
sql_vw_sla_pivot = 'select * from vw_sla_code_pivot'

## Execute all of the sql queries and store the results

In [9]:
month_tbl  = pd.read_sql(con = engine, sql = sql_month_tbl)

In [11]:
before_sla_season = pd.read_sql(con = engine, sql = sql_sla_season)

In [12]:
date_cols_season = [
    'effective_start', 'effective_end', 'created_date_utc', 'updated_date_utc',
    'effective_start_adj', 'effective_end_adj'
]

In [13]:
for i in date_cols_season:
    before_sla_season[i] = pd.to_datetime(before_sla_season[i])

In [14]:
before_sla_season.head()

,season_id,season_desc,year_round,effective,effective_end,effective_start,created_date_utc,updated_date_utc,effective_start_adj,effective_end_adj
0,1,"Year-round, not seasonal",True,False,2020-07-24,2014-01-01,2020-03-03 19:12:12.710,2020-07-28 20:16:56.850,2014-01-05,2020-07-25
1,2,"Beaches, etc.",False,False,2020-07-25,2019-07-01,2020-03-03 19:12:12.723,2020-07-29 12:39:31.760,2019-07-07,2020-07-25
2,3,"Ballfields, etc.",False,False,2020-07-25,2019-07-01,2020-03-03 19:12:12.723,2020-07-29 12:39:31.760,2019-07-07,2020-07-25
3,4,"NY Pause, COVID-19 Pandemic",True,False,2020-07-04,2020-03-22,2020-04-20 21:52:46.403,2020-07-01 15:20:36.193,2020-03-22,2020-07-04
4,6,New year round test season.,True,True,NaT,2020-07-24,2020-07-24 23:41:46.543,2020-07-28 18:31:55.867,2020-07-26,NaT


In [15]:
before_change_request = pd.read_sql(con = engine, sql = sql_change_request)

In [16]:
before_change_request.head()

,change_request_id,unit_id,sla_code,season_id,effective_start,change_request_justification,effective_start_adj,sla_change_status
0,108462,M299,16,1,2019-07-01,AMPS updates since original review,2019-07-07,2
1,108463,M300,1,1,2014-01-01,Original Review,2014-01-05,2
2,108464,M301,1,1,2014-01-01,Original Review,2014-01-05,2
3,108465,M302,6,1,2014-01-01,Original Review,2014-01-05,2
4,108466,M303,1,1,2014-01-01,Original Review,2014-01-05,2


In [17]:
date_cols_change_req = ['effective_start', 'effective_start_adj']

In [18]:
for i in date_cols_change_req:
    before_change_request[i] = pd.to_datetime(before_change_request[i])

In [19]:
before_change_request_status = pd.read_sql(con = engine, sql = sql_change_request_status)

In [21]:
before_change_request_status.head()

,change_request_status_id,change_request_id,sla_change_status,created_date_utc,created_user
0,1,114208,1,2020-03-03 04:00:00,0000000
1,2,114207,1,2020-03-03 04:00:00,0000000
2,3,114206,1,2020-03-03 04:00:00,0000000
3,4,114205,1,2020-03-03 04:00:00,0000000
4,5,114204,1,2020-03-03 04:00:00,0000000


In [23]:
date_cols_change_req_stat = ['created_date_utc']

In [24]:
for i in date_cols_change_req_stat:
    before_change_request_status[i] = pd.to_datetime(before_change_request_status[i])

In [25]:
before_unit_season = pd.read_sql(con = engine, sql = sql_unit_sla_season)
before_unit_season.head()

,sla_season_id,unit_id,sla_code,season_id,effective,effective_start,effective_end,change_request_id,created_date_utc,updated_date_utc,effective_start_adj,effective_end_adj
0,1,-,16,1,False,2019-07-01,2020-07-24,109721,NaT,NaT,2019-07-07,2020-07-25
1,2,11772,16,1,False,2019-07-01,2020-07-24,109722,NaT,NaT,2019-07-07,2020-07-25
2,3,12654,16,1,False,2019-07-01,2020-07-24,109723,NaT,NaT,2019-07-07,2020-07-25
3,4,14301,16,1,False,2019-07-01,2020-07-24,109724,NaT,NaT,2019-07-07,2020-07-25
4,5,14918,16,1,False,2019-07-01,2020-07-24,109725,NaT,NaT,2019-07-07,2020-07-25


In [26]:
date_cols_unit_season = [
    'effective_start', 'effective_end', 'created_date_utc', 'updated_date_utc',
    'effective_start_adj', 'effective_end_adj'
]

In [27]:
for i in date_cols_unit_season:
    before_unit_season[i] = pd.to_datetime(before_unit_season[i])

In [21]:
timing = []

In [22]:
descriptions = []

In [23]:
results = []

## Before Tests

### First before test

In [24]:
timing.append('before')

In [25]:
descriptions.append('''Ensure that 'effective'==1 for every season that has\
'effective_end_date'==null''')

In [29]:
before_change_request.head()

,change_request_id,unit_id,sla_code,season_id,effective_start,change_request_justification,effective_start_adj,sla_change_status
0,108462,M299,16,1,2019-07-01,AMPS updates since original review,2019-07-07,2
1,108463,M300,1,1,2014-01-01,Original Review,2014-01-05,2
2,108464,M301,1,1,2014-01-01,Original Review,2014-01-05,2
3,108465,M302,6,1,2014-01-01,Original Review,2014-01-05,2
4,108466,M303,1,1,2014-01-01,Original Review,2014-01-05,2


In [36]:
temp_df = before_change_request[[
    'change_request_id', 'effective_start', 'effective_start_adj'
]].merge(before_unit_season[[
    'change_request_id', 'effective_start', 'effective_start_adj',
    'effective_end', 'effective_end_adj', 'created_date_utc'
]], on = 'change_request_id')
temp_df.head()

,change_request_id,effective_start_x,effective_start_adj_x,effective_start_y,effective_start_adj_y,effective_end,effective_end_adj,created_date_utc
0,108462,2019-07-01,2019-07-07,2019-07-01,2019-07-07,2020-07-24,2020-07-25,NaT
1,108463,2014-01-01,2014-01-05,2014-01-01,2014-01-05,2020-03-21,2020-03-21,NaT
2,108464,2014-01-01,2014-01-05,2014-01-01,2014-01-05,2020-04-04,2020-04-04,NaT
3,108465,2014-01-01,2014-01-05,2014-01-01,2014-01-05,2020-03-21,2020-03-21,NaT
4,108466,2014-01-01,2014-01-05,2014-01-01,2014-01-05,2020-04-04,2020-04-04,NaT


In [38]:
temp_df[pd.isnull(temp_df['effective_end_adj'])]

,change_request_id,effective_start_x,effective_start_adj_x,effective_start_y,effective_start_adj_y,effective_end,effective_end_adj,created_date_utc
12318,160881,2020-07-24,2020-07-26,2020-07-24,2020-07-26,NaT,NaT,NaT
12319,160882,2020-07-24,2020-07-26,2020-07-24,2020-07-26,NaT,NaT,NaT
12320,160883,2020-07-24,2020-07-26,2020-07-24,2020-07-26,NaT,NaT,NaT
12321,160884,2020-07-24,2020-07-26,2020-07-24,2020-07-26,NaT,NaT,NaT
12322,160885,2020-07-24,2020-07-26,2020-07-24,2020-07-26,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...
18149,166998,2020-08-23,2020-08-23,2020-08-23,2020-08-23,NaT,NaT,2020-08-24 15:01:02.143
18150,167002,2020-08-23,2020-08-23,2020-08-23,2020-08-23,NaT,NaT,2020-08-24 15:52:46.910
18151,167003,2020-08-23,2020-08-23,2020-08-23,2020-08-23,NaT,NaT,2020-08-24 15:52:46.910
18152,167004,2020-08-23,2020-08-23,2020-08-23,2020-08-23,NaT,NaT,2020-08-24 15:52:46.910


In [33]:
before_unit_season.head()

,sla_season_id,unit_id,sla_code,season_id,effective,effective_start,effective_end,change_request_id,created_date_utc,updated_date_utc,effective_start_adj,effective_end_adj
0,1,-,16,1,False,2019-07-01,2020-07-24,109721,NaT,NaT,2019-07-07,2020-07-25
1,2,11772,16,1,False,2019-07-01,2020-07-24,109722,NaT,NaT,2019-07-07,2020-07-25
2,3,12654,16,1,False,2019-07-01,2020-07-24,109723,NaT,NaT,2019-07-07,2020-07-25
3,4,14301,16,1,False,2019-07-01,2020-07-24,109724,NaT,NaT,2019-07-07,2020-07-25
4,5,14918,16,1,False,2019-07-01,2020-07-24,109725,NaT,NaT,2019-07-07,2020-07-25


In [26]:
results.append(
    len(before_sla_season[(pd.isnull(before_sla_season['effective_end']))
                          & (before_sla_season['effective'] == False)]) == 0)

### Second before test

In [27]:
timing.append('before')

In [28]:
descriptions.append('''Test that 'effective_end_adj' is after today if \
'effective'==1 AND 'effective_end' != null in tbl_sla_season''')

In [29]:
before_sla_season['effective_end_adj'] = pd.to_datetime(
    before_sla_season['effective_end_adj'])

In [30]:
today = datetime.datetime.today()

In [31]:
temp_df = before_sla_season[(before_sla_season['effective'] == True)
                  & (~pd.isnull(before_sla_season['effective_end']))].copy()

In [32]:
results.append(len(temp_df[temp_df['effective_end_adj'] > today])==len(temp_df))

### Third before test

In [33]:
timing.append('before')

In [34]:
descriptions.append('''Test that every season in tbl_sla_season has exactly two\
entries in tbl_ref_sla_season_definition''')

In [35]:
entry_count = []
for i in before_sla_season.season_id.values:
    entry_count.append(
        len(before_season_definition[before_season_definition['season_id'] ==
                                     i]) == 2)

In [36]:
results.append(False not in entry_count)

### Fourth before test

In [37]:
timing.append('before')

In [38]:
descriptions.append('''Test that in tbl_ref_sla_season_definition if 'date_ref_fixed'==True,\
then columns 'month_name_desc' and 'date_ref_day_number' are not null ''')

In [39]:
temp_df = before_season_definition[before_season_definition['date_ref_fixed']
                                   == True]

In [40]:
results.append(
    len(temp_df) == len(temp_df[~(pd.isnull(temp_df['month_name_desc']))
                                &
                                (~pd.isnull(temp_df['date_ref_day_number']))]))

### Fifth before test

In [41]:
timing.append('before')

In [42]:
descriptions.append('''Test that in tbl_ref_sla_season_definition if 'date_ref_fixed'==False,\
then columns 'month_name_desc' and 'day_name_desc' and 'day_rank_id' are not null ''')

In [43]:
temp_df = before_season_definition[before_season_definition['date_ref_fixed']==False]

In [44]:
results.append(
    len(temp_df) == len(temp_df[~(pd.isnull(temp_df['month_name_desc']))
                                & (~pd.isnull(temp_df['day_name_desc']))
                                & (~pd.isnull(temp_df['day_rank_id']))]))

### Sixth before test

In [45]:
timing.append('before')

In [46]:
descriptions.append(
    '''For year-round seasons, test that 'date_ref_fixed'==1,\
    and start/end dates are Jan 1/Dec 31 ''')

In [47]:
temp_df = before_season_definition.merge(
    before_sla_season[before_sla_season['year_round'] == True],
    on='season_id')[[
        'season_id', 'year_round', 'date_ref_fixed', 'month_name_desc',
        'date_ref_day_number', 'date_type_id'
    ]]
temp_df

,season_id,year_round,date_ref_fixed,month_name_desc,date_ref_day_number,date_type_id
0,1,True,True,January,1.0,1
1,1,True,True,December,31.0,2
2,4,True,True,January,1.0,1
3,4,True,True,December,31.0,2
4,6,True,True,January,1.0,1
5,6,True,True,December,31.0,2
6,20,True,True,January,1.0,1
7,20,True,True,December,31.0,2
8,23,True,True,January,1.0,1
9,23,True,True,December,31.0,2


In [48]:
len(temp_df)==len(temp_df[((temp_df['date_type_id'] == 1)
         & (temp_df['month_name_desc'] == 'January')
         & (temp_df['date_ref_day_number'] == 1)) &
        (temp_df['date_ref_fixed'] == 1) |
        ((temp_df['date_type_id'] == 2)
         & (temp_df['month_name_desc'] == 'December')
        & (temp_df['date_ref_day_number'] == 31)) &
        (temp_df['date_ref_fixed'] == 1)])

True

In [49]:
results.append(
    len(temp_df) == len(temp_df[((temp_df['date_type_id'] == 1)
                                 & (temp_df['month_name_desc'] == 'January')
                                 & (temp_df['date_ref_day_number'] == 1))
                                & (temp_df['date_ref_fixed'] == 1)
                                | ((temp_df['date_type_id'] == 2)
                                   & (temp_df['month_name_desc'] == 'December')
                                   & (temp_df['date_ref_day_number'] == 31))
                                & (temp_df['date_ref_fixed'] == 1)]))

### Seventh before test

In [50]:
timing.append('before')

In [51]:
# before_season_definition

In [52]:
descriptions.append(
    '''For periodic seasons, test that the difference between start/end months is\
    between 2 and 10 inclusive ''')

In [53]:
temp_df = before_season_definition.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id', 'year_round', 'date_ref_fixed', 'month_name_desc',
        'date_ref_day_number', 'date_type_id'
    ]]
temp_df

,season_id,year_round,date_ref_fixed,month_name_desc,date_ref_day_number,date_type_id
0,2,False,True,May,1.0,1
1,2,False,True,October,1.0,2
2,3,False,True,March,1.0,1
3,3,False,True,October,31.0,2
4,10,False,True,May,1.0,1
5,10,False,True,October,1.0,2
6,9,False,True,March,1.0,1
7,9,False,True,October,31.0,2
8,12,False,True,May,1.0,1
9,12,False,True,October,1.0,2


In [54]:
temp_df = before_season_definition.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id',  'month_name_desc',
        'date_type_id'
    ]].merge(month_tbl[['month_name_desc','month_name_num']], on='month_name_desc')
temp_df

,season_id,month_name_desc,date_type_id,month_name_num
0,2,May,1,5
1,10,May,1,5
2,12,May,1,5
3,16,May,1,5
4,18,May,2,5
5,21,May,1,5
6,24,May,1,5
7,2,October,2,10
8,3,October,2,10
9,10,October,2,10


In [55]:
temp_df2 = temp_df.sort_values(by='date_type_id').groupby('season_id').first().merge(
    temp_df.sort_values(by='date_type_id').groupby('season_id').last(),
    left_index=True,
    right_index=True)[['month_name_num_x',
                       'month_name_num_y']].rename(columns={
                           'month_name_num_x': 'start',
                           'month_name_num_y': 'end'
                       })
temp_df2

,start,end
season_id,,
2,5,10
3,3,10
9,3,10
10,5,10
11,3,10
12,5,10
14,4,11
16,5,9
17,4,9


In [56]:
temp_df2 = temp_df.sort_values(by='date_type_id').groupby('season_id').first().merge(
    temp_df.sort_values(by='date_type_id').groupby('season_id').last(),
    left_index=True,
    right_index=True)[['month_name_num_x',
                       'month_name_num_y']].rename(columns={
                           'month_name_num_x': 'start',
                           'month_name_num_y': 'end'
                       })

In [57]:
def get_diff(row, col_strt, col_end):
    if row[col_end]-row[col_strt]>0:
        return row[col_end]-row[col_strt]
    elif row[col_end]-row[col_strt]<0:
        return 12 + row[col_end]-row[col_strt]

In [58]:
temp_df2['diff'] = temp_df2.apply(lambda row: get_diff(row, 'start', 'end'), axis = 1)

In [59]:
range_vals = []
for i in temp_df2['diff'].values:
    if (i < 2) | (i > 10):
        range_vals.append(False)
    else:
        range_vals.append(True)           

In [60]:
# range_vals

In [61]:
results.append(False not in range_vals)

### Eighth before test

In [62]:
timing.append('before')

In [63]:
descriptions.append(
    '''For year-round seasons, test that 'effective_start' in tbl_sla_season_date\
    is equal to 'effective_start' in tbl_sla_season ''')

In [64]:
temp_df = before_season_date.merge(
    before_sla_season[before_sla_season['year_round'] == True],
    on='season_id')[[
        'season_id', 'year_round', 'effective_start_x', 'effective_start_y'
    ]]
temp_df

,season_id,year_round,effective_start_x,effective_start_y
0,1,True,2014-01-01,2014-01-01
1,4,True,2020-03-22,2020-03-22
2,6,True,2020-07-24,2020-07-24
3,20,True,2020-08-09,2020-08-09
4,23,True,2020-08-16,2020-08-16


In [65]:
results.append(
    len(temp_df) == len(temp_df[temp_df['effective_start_x'] ==
                                temp_df['effective_start_y']]))

### Ninth before test

In [66]:
timing.append('before')

In [67]:
descriptions.append(
    '''For periodic seasons, test that the earliest 'effective_start' for each season\
    in tbl_sla_season_date is equal to 'effective_start' in tbl_sla_season ''')

In [68]:
temp_df = before_season_date.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id', 'year_round', 'effective_start_x', 'effective_start_y'
    ]].sort_values(by='effective_start_x').groupby('season_id').first()
temp_df

,year_round,effective_start_x,effective_start_y
season_id,,,
2,False,2019-07-01,2019-07-01
3,False,2019-07-01,2019-07-01
9,False,2020-07-26,2020-07-26
10,False,2020-07-26,2020-07-26
11,False,2020-08-09,2020-08-09
12,False,2020-08-07,2020-08-07
14,False,2020-08-13,2020-08-13
16,False,2020-08-15,2020-08-15
17,False,2020-08-16,2020-08-16


In [69]:
results.append(
    len(temp_df) == len(temp_df[temp_df['effective_start_x'] ==
                                temp_df['effective_start_y']]))

### Tenth before test

In [70]:
timing.append('before')

In [71]:
descriptions.append(
    '''For periodic seasons, test that the latest 'effective_end' for each season\
    in tbl_sla_season_date is equal to 'effective_end' in tbl_sla_season OR today+1year (if null) ''')

In [72]:
temp_df = before_season_date.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id', 'year_round', 'effective_start_x', 'effective_end_x', 'effective_end_y'
    ]].sort_values(by='effective_start_x').groupby('season_id').last()
temp_df

,year_round,effective_start_x,effective_end_x,effective_end_y
season_id,,,,
2,False,2020-05-01,2020-07-25,2020-07-25
3,False,2020-03-01,2020-07-25,2020-07-25
9,False,2020-07-26,2020-08-08,2020-08-08
10,False,2020-07-26,2020-08-08,2020-08-08
11,False,2021-03-01,2021-09-04,NaT
12,False,2020-08-07,2020-08-15,2020-08-15
14,False,2021-04-15,2021-09-04,NaT
16,False,2021-05-31,2021-09-04,NaT
17,False,2021-04-15,2021-09-04,NaT


In [73]:
two_conditions = []

In [74]:
two_conditions.append(len(temp_df[~pd.isnull(temp_df['effective_end_y'])]) == len(
    temp_df[(~pd.isnull(temp_df['effective_end_y']))
            & (temp_df['effective_end_x'] == temp_df['effective_end_y'])]))

In [75]:
two_conditions.append(
    len(temp_df[pd.isnull(temp_df['effective_end_y'])]) == len(temp_df[
        (pd.isnull(temp_df['effective_end_y']))
        & (temp_df['effective_end_x'] == datetime.datetime(
            today.year + 1, today.month, today.day))]))

In [76]:
results.append(False not in two_conditions)

### Eleventh before test

In [77]:
timing.append('before')

In [78]:
descriptions.append(
    '''Test to make sure there are no gaps in dates in tbl_sla_season_date ''')

In [83]:
sql_gaps_1 = '''
select season_id,
       datediff(day, min(effective_start), max(effective_end)) as daydiff
from sladb.dbo.tbl_sla_season_date
group by season_id
'''

sql_gaps_2 = '''select season_id,
       case when count(*) > 1 then sum(daydiff) + (count(*) - 1)
            else sum(daydiff)
       end as daydiff
from (select season_id,
             datediff(day, effective_start, effective_end) as daydiff
      from sladb.dbo.tbl_sla_season_date) as t
group by season_id
'''

In [84]:
gaps_df1 = pd.read_sql(con = engine, sql = sql_gaps_1)
gaps_df2 = pd.read_sql(con = engine, sql = sql_gaps_2)

In [85]:
temp_df = gaps_df1.merge(gaps_df2, on='season_id')

In [86]:
results.append(len(temp_df)==len(temp_df[temp_df['daydiff_x']==temp_df['daydiff_y']]))

## Make Updates

## Execute all of the sql queries AFTER UPDATES and store the results

## List of tuples with final test results

In [75]:
# list(zip(timing, descriptions, results))

In [87]:
pd.DataFrame(list(zip(timing, descriptions, results)), columns = {'before/after', 'test_desc', 'result'})             

,test_desc,result,before/after
0,before,Ensure that 'effective'==1 for every season th...,True
1,before,Test that 'effective_end_adj' is after today i...,True
2,before,Test that every season in tbl_sla_season has e...,True
3,before,Test that in tbl_ref_sla_season_definition if ...,True
4,before,Test that in tbl_ref_sla_season_definition if ...,True
5,before,"For year-round seasons, test that 'date_ref_fi...",True
6,before,"For periodic seasons, test that the difference...",True
7,before,"For year-round seasons, test that 'effective_s...",True
8,before,"For periodic seasons, test that the earliest '...",True
9,before,"For periodic seasons, test that the latest 'ef...",True
